In [277]:
from torchvision.datasets import FashionMNIST
from torch.utils.data import DataLoader, TensorDataset

train = FashionMNIST('.', download=True, train=True)
n = len(train)
valid_n = n * 3 // 10
cuda = torch.device("cuda")
cpu = torch.device("cpu")
dev = cpu #cuda if torch.cuda.is_available() else cpu

@contextmanager
def switch_to_cpu(model):
    global dev
    _dev = dev
    dev = cpu
    model.to(cpu)
    yield
    dev = _dev
    model.to(dev)

def preprocess(x, y):
    return x.view(-1, 1, 28, 28).to(dev), y.to(dev)

class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func

    def __len__(self):
        return len(self.dl)

    def __iter__(self):
        for b in self.dl:
            yield (self.func(*b))

def get_dl(xs, ys, bs=30):
    return WrappedDataLoader(DataLoader(TensorDataset(xs.type(torch.float32), ys), batch_size=bs), preprocess)

valid_dl = get_dl(train.train_data[:valid_n], train.train_labels[:valid_n])
train_dl = get_dl(train.train_data[valid_n:], train.train_labels[valid_n:])

torch.manual_seed(2137)
torch.cuda.manual_seed_all(213742069)
torch.cuda.manual_seed(213742069)

In [253]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score

# The following is adapted from:
# https://pytorch.org/tutorials/beginner/nn_tutorial.html

class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x):
        return self.func(x)


def get_model():
    return nn.Sequential(
        nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),
        nn.ReLU(),
        nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1),
        nn.ReLU(),
        nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1),
        nn.AvgPool2d(4),
        Lambda(lambda x: x.view(x.size(0), -1)),
    )
    

lr = 0.1

def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

def accuracy(model, valid_dl):
    reals = []
    preds = []
    for xb, yb in valid_dl:
        reals.append(yb)
        preds.append(model(xb).argmax(dim=1))
    
    return accuracy_score(torch.cat(reals, 0), torch.cat(preds, 0))

def fit(model, loss_func, opt, train_dl, valid_dl):
    last_loss = 2137.
    epoch = 1
    while True:
        model.train()
        for xb, yb in train_dl:
#             print(model(xb).size())
#             assert False
            loss_batch(model, loss_func, xb, yb, opt)

        model.eval()
        with torch.no_grad(), switch_to_cpu(model):
            # print diagnostics about current epoch
            losses, nums = zip(*[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl])
            val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

            print("Epoch %s: val_loss=%s, val_acc=%s, train_acc=%s" % (epoch, val_loss, accuracy(model, valid_dl), accuracy(model, train_dl)))
        if last_loss - val_loss < 0.00001:
            print("delta loss was %s, stopping" % (last_loss - val_loss))
            break
        last_loss = val_loss
        epoch += 1

def initer(layer):
    if type(layer) == nn.Conv2d:
        nn.init.kaiming_normal_(layer.weight)

model = get_model()
model.to(dev)
model.apply(initer)
fit(model, F.cross_entropy, torch.optim.Adam(model.parameters()), train_dl, valid_dl)
print("DONE!")
# SGD : val_loss=0.4727671396235625,  val_acc=0.8349444444444445, train_acc=0.8478333333333333
# Adam: val_loss=0.43147962642212706, val_acc=0.8516666666666667, train_acc=0.8738809523809524
# Seq : val_loss=0.43787566319108007, val_acc=0.8512222222222222, train_acc=0.8717142857142857
# cuda: val_loss=0.4572788599754373,  val_acc=0.8385555555555556, train_acc=0.8583095238095239
# 2137: val_loss=0.4622961250692606,  val_acc=0.8357777777777777, train_acc=0.8569761904761904
# cpu : stride=1 -> not worth trying
#     : val_loss=0.5751239460210006,  val_acc=0.7962777777777778, train_acc=0.8186904761904762 - maxpool
#     : val_loss=0.4394427917525172,  val_acc=0.8492222222222222, train_acc=0.8715238095238095 - early
#     : val_loss=0.43841269160310425, val_acc=0.8505,             train_acc=0.8701428571428571 - real early

Epoch 1: val_loss=0.7282379469027122, val_acc=0.7547777777777778, train_acc=0.7592380952380953
Epoch 2: val_loss=0.6146034619460503, val_acc=0.7928888888888889, train_acc=0.8004523809523809
Epoch 3: val_loss=0.5893327518055836, val_acc=0.8017222222222222, train_acc=0.8106904761904762
Epoch 4: val_loss=0.552944111811618, val_acc=0.8130555555555555, train_acc=0.8225
Epoch 5: val_loss=0.5292572170123457, val_acc=0.8187777777777778, train_acc=0.8302857142857143
Epoch 6: val_loss=0.4999961622804403, val_acc=0.8269444444444445, train_acc=0.8389047619047619
Epoch 7: val_loss=0.48341955239574114, val_acc=0.8317222222222223, train_acc=0.8435714285714285
Epoch 8: val_loss=0.4677774465829134, val_acc=0.8367777777777777, train_acc=0.8498095238095238
Epoch 9: val_loss=0.4566389107579986, val_acc=0.8404444444444444, train_acc=0.853547619047619
Epoch 10: val_loss=0.45491250564654667, val_acc=0.8401666666666666, train_acc=0.8543333333333333
Epoch 11: val_loss=0.44485253871728975, val_acc=0.84472222222

In [232]:
torch.cuda.device_count()

1

In [37]:
import torch
from skorch import NeuralNetClassifier
net_regr = NeuralNetClassifier(
    module=FirstCNN,
    max_epochs=20,
    lr=0.1,
    criterion=nn.CrossEntr,
#     criterion__weight=weight,
    optimizer=torch.optim.Adam,
    optimizer__momentum=0.9,
#     device='cuda',  # uncomment this to train with CUDA
)

In [30]:
train.train_labels.unique()

tensor([1, 9, 0, 3, 2, 7, 5, 6, 4, 8])

In [34]:
train.train_data = train.train_data.type('torch.FloatTensor')
net_regr.fit(train.train_data, train.train_labels)

  epoch    train_loss    valid_acc    valid_loss      dur
-------  ------------  -----------  ------------  -------
      1           inf       0.1000      -16.1242  15.5629
      2      -16.6445       0.1000      -17.0698  15.5948
      3      -17.3730       0.1004      -17.6433  15.3967
      4      -17.8578       0.1019      -18.0565  15.9598
      5      -18.2226       0.1047      -18.3799  15.4213
      6      -18.5153       0.1094      -18.6457  15.9111
      7      -18.7600       0.1146      -18.8713  15.4875
      8      -18.9701       0.1199      -19.0674  15.7156
      9      -19.1544       0.1279      -19.2407  15.3496
     10      -19.3184       0.1368      -19.3960  15.3957
     11      -19.4662       0.1431      -19.5367  15.3407
     12      -19.6007       0.1486      -19.6653  15.9689
     13      -19.7241       0.1552      -19.7838  15.3812
     14      -19.8381       0.1616      -19.8936  15.3919
     15      -19.9441       0.1651      -19.9959  15.3365
     16      -

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=FirstCNN(
    (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (conv3): Conv2d(16, 10, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  ),
)

In [278]:
test = FashionMNIST('.', download=True, train=False)
test_dl = get_dl(test.test_data, torch.ones(test.test_data.size(0)), 10000)
# preds = model()
next(iter(test_dl))[0].size()

torch.Size([10000, 1, 28, 28])

In [139]:
preds = preds.argmax(dim=1)

In [140]:
import pandas as pd
df = pd.DataFrame()
df['Class'] = preds
df.index.name = 'Id'
df.to_csv('submission.csv')
df

,Class
Id,
0,9
1,2
2,1
3,1
4,6
5,1
6,4
7,6
8,5
